# Likelihood Optimization of gas Kinematics in IFUs (LOKI)
## Fitting example: MIRI + QSO model

Michael Reefe

This example notebook provides a tutorial on how to run LOKI on a multi-channel MIRI/MRS cube.  This is a continuation of "example_MIRI_psf_model" -- be sure to run that one first!  This will take the results of that fit and reconstruct a 1D QSO spectrum from the fitted AGN PSF template.  Then, we'll fit a model to the QSO spectrum itself.

We won't need multi-processing for this example, since we're just fitting one spectrum.

In [1]:
using Pkg
Pkg.activate(dirname(@__DIR__))
Pkg.instantiate()
Pkg.precompile()
using Loki
using FITSIO

  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/Loki`


Since we've already created a pre-processed MIRI data cube in the previous example, we can just load it in here without redoing any of that work.

In [2]:
channel = 0   # since we are including data from all 4 channels, this is just a placeholder
z = 0.016317
nm = "NGC_7469"
obs = from_fits(["$nm.channel$channel.rest_frame.fits"], z);
run_name = "$(nm)_ch$(channel)_qso_model"

[ Info: Initializing DataCube struct from NGC_7469.channel0.rest_frame.fits


"NGC_7469_ch0_qso_model"

We will also need to read in two fits files that should have been produced as outputs in the previous example.  The "\*_parameter_maps.fits" which contains information on the fitted amplitudes of the PSF model, and the "\*_full_model.fits" which contains the PSF model itself.

In [3]:
hdu_param = FITS("output_NGC_7469_ch0_psf_model/NGC_7469_ch0_psf_model_parameter_maps.fits");
hdu_model = FITS("output_NGC_7469_ch0_psf_model/NGC_7469_ch0_psf_model_full_model.fits");

Next, we create a `CubeFitter` as usual, but this time we don't include the "templates" or "template_names" arguments.  Instead, we want to tailor the CubeFitter to model the AGN spectrum, so we've enabled the "fit_sil_emission" component.

Instead of calling "fit_cube!", we use a different function, "post_fit_nuclear_template!", which takes as arguments the CubeFitter, the two fits files that we read in above, and the name of the AGN PSF template that was used in the previous run.

This is also where we'll show off the bootstrapping capabilities of the code! All we have to do is add the argument "n_bootstrap" to specify some number of bootstrapping iterations.  In general it's good to do bootstrapping on single-spectrum fits like this.  But when you're fitting a whole cube, such as in the previous example, adding bootstrapping can greatly increase the runtime.  If you want to check on the progress at any time, you can look at the log file located under output_NGC_7469_ch0_qso_model/logs/loki.spaxel_1_1.log.

In [4]:
# To see a full list of keyword arguments, please refer to the docstring, which can be accessed by typing `?CubeFitter` in the command
# line after importing Loki.
cube_fitter = CubeFitter(
    obs.channels[channel], 
    obs.z, 
    run_name; 
    parallel=false, 
    plot_spaxels=:pyplot, 
    plot_maps=true, 
    save_fits=true,
    silicate_absorption="d+",
    extinction_screen=true, 
    use_pah_templates=true, 
    fit_sil_emission=true, 
    fit_stellar_continuum=false, 
    save_full_model=true, 
    map_snr_thresh=3., 
    # templates=nuc_temp, 
    # template_names=["nuclear"], 
    subtract_cubic_spline=true,
    n_bootstrap=100
)

post_fit_nuclear_template!(cube_fitter, hdu_param, hdu_model, "NUCLEAR")

[ Info: Preparing output directories
┌ Info: 
│ 
└ BEGINNING NUCELAR TEMPLATE FITTING ROUTINE FOR NGC_7469_ch0_qso_model
[ Info: ===> Preparing output data structures... <===
[ Info: Integrating spectrum across the whole cube...
[ Info: ===> Beginning nuclear spectrum fitting... <===
[ Info: ===> Generating parameter maps and model cubes... <===
[ Info: ===> Plotting QSO parameter maps... <===
┌ Warning: You have requested combined plots for the lines [:OII_3727A, :OII_3729A], but at least one of these lines was not fit!
└ @ Loki ~/Dropbox/Astrophysics/Phoenix_Cluster/Loki/src/core/output.jl:634
[ Info: ===> Writing 1D FITS outputs... <===
[ Info: ===> Writing 3D FITS outputs... <===
[ Info: Done!!


(CubeFitter{Float64, Int64, Unitful.Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, Unitful.Quantity{Float64, 𝐋 𝐓⁻¹, Unitful.FreeUnits{(km, s⁻¹), 𝐋 𝐓⁻¹, nothing}}, Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}(DataCube{Vector{Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}, Array{Unitful.Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, 3}}(Unitful.Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}[4.822511180406425 μm, 4.823298464867244 μm, 4.824085877853802 μm, 4.824873419387084 μm, 4.825661089488074 μm, 4.826448888177762 μm, 4.82723681547714 μm, 4.8280248714072025 μm, 4.828813055988951 μm, 4.8296013692433855 μm  …  17.17219430353727 μm, 17.174997698123118 μm, 17.17780155036866 μm, 17.18060586034861 μm, 17.183410628137686 μm, 17.186215853810637 μm, 17.18902153744222 μm, 17.19182767910719 μm, 17.194634278880308 μm, 17.197441336836377 μm], Unitful.Quanti

And the results can be found in the "output_[run_name]" directory, just like the other examples!  This is what the final fit looks like:

![](./NGC7469.quasar.png)

The orange line shows the final model.  The decomposed components of the model consist of:
- Thermal dust continuum, in gray
- The hot silicate dust emission, in light green
- PAHs, in blue
- Emission lines, in purple
- Extinction, in dotted gray (read from the right axis)

Since we used bootstrapping for this run, the minimum/maximum range of models is shown with the orange shaded region (though its a bit hard to see in this example).